<a href="https://colab.research.google.com/github/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/05-calculating-loss/cross_entropy_loss_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cross-Entropy Loss from Scratch

To train a model, we tweak the weights and biases to improve the model’s accuracy and confidence. To do this, we calculate how much error the model has. The loss function , also referred to as the cost function , is the algorithm that quantifies how wrong a model is. Loss is the measure of this metric. Since loss is the model’s error, we ideally want it to be 0.

The output of a neural network is actually confidence, and more confidence in the correct answer is better. Because of this, we strive to increase correct confidence and decrease misplaced confidence.

##Setup

In [9]:
import numpy as np
import math

## Categorical Cross-Entropy Loss

If you’re familiar with linear regression, then you already know one of the loss functions used with neural networks that do regression: squared error (or mean squared error with neural networks).

The model has a softmax activation function for the output layer, which means it’s outputting a probability distribution. Categorical cross-entropy is explicitly used to compare a `ground-truth` probability ( y or `targets`) and some predicted distribution ( y-hat or `predictions`), so it makes sense to use cross-entropy here. It is also one of the most commonly used loss functions with a softmax activation on the output layer.

The formula for calculating the categorical cross-entropy of `y` (actual/desired distribution) and `y-hat` (predicted distribution) is:

$$
L_i = - \sum_j y_{i, j}log(\hat{y_{i, j}})
$$

In general, the log loss error function is what we apply to the output of a binary logistic regression model — there are only two classes in the distribution, each of them applying to a single output (neuron) which is targeted as a `0` or `1`. In our case, we have a classification model that returns a probability distribution over all of the outputs. Cross-entropy
compares two probability distributions. 

In our case, we have a softmax output, let’s say it’s:

In [2]:
softmax_output = [0.7, 0.1, 0.2]

We have 3 class confidences in the above output, and let’s assume that the desired prediction is the first class (index 0, which is currently 0.7). If that’s the intended prediction, then the desired probability distribution is `[1, 0, 0]`.

Arrays or vectors like this are called one-hot , meaning one of the values is “hot” (on), with a value of 1, and the rest are “cold” (off), with values of 0. When comparing the model’s results to a one-hot vector using cross-entropy, the other parts of the equation zero out, and the target probability’s log loss is
multiplied by 1, making the cross-entropy calculation relatively simple. This is also a special case of the cross-entropy calculation, called categorical cross-entropy. 

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/neural-networks-from-scratch/05-calculating-loss/images/1.png?raw=1' width='800'/>

To exemplify this — if we take a softmax output of `[0.7, 0.1, 0.2]` and targets of `[1, 0, 0]` , we can apply the calculations as follows:

In [10]:
L = -(1 * math.log(0.7) + 0 * math.log(0.1) + 0 * math.log(0.2))
print(L)

0.35667494393873245


In [8]:
L = -(-0.35667494393873245 + 0  + 0)
print(L)

0.35667494393873245


Let’s see the Python code for this:

In [11]:
# An example output from the output layer of the neural network
softmax_output = [0.7, 0.1, 0.2]
# Ground truth
target_output = [1, 0, 0]

loss = -(math.log(softmax_output[0]) * target_output[0] +
         math.log(softmax_output[1]) * target_output[1] +
         math.log(softmax_output[2]) * target_output[2])

print(loss)

0.35667494393873245


That’s the full categorical cross-entropy calculation, but we can make a few assumptions given one-hot target vectors.

The example confidence level might look like `[0.22, 0.6, 0.18]` or `[0.32, 0.36, 0.32]`. In both cases, the argmax of these vectors will return the second
class as the prediction, but the model’s confidence about these predictions is high only for one of them. The Categorical Cross-Entropy Loss accounts for that and outputs a larger loss the lower the confidence is:

In [12]:
print (math.log( 1. ))
print (math.log( 0.95 ))
print (math.log( 0.9 ))
print (math.log( 0.8 ))
print ( '...' )
print (math.log( 0.2 ))
print (math.log( 0.1 ))
print (math.log( 0.05 ))
print (math.log( 0.01 ))

0.0
-0.05129329438755058
-0.10536051565782628
-0.2231435513142097
...
-1.6094379124341003
-2.3025850929940455
-2.995732273553991
-4.605170185988091


When the confidence level equals 1 , meaning the model is 100% “sure” about its prediction, the loss value for this sample equals 0 . The loss value raises with the confidence level, approaching 0. You might also wonder
why we did not print the result of log(0) — we’ll explain that shortly.